Проверка на наличие GPU и подключение CUDA

In [1]:
%%bash
pip install pycuda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pycuda: filename=pycuda-2022.2-cp38-cp38-linux_x86_64.whl size=646470 sha256=ba6ca42349b3b3731061cc9a07eff76a8becfcd29913404e530f74a0b0e1db34
  Stored in directory: /root/.cache/pip/wheels/ef/0e/11/809b20ee6df2e1c63ef52b3dcf2b7386b900d4f58548f0484e
  Created wheel for pytools: filename=pytools-2022.1.13-py2.py3-none-any.whl size=66024 sha256=feedc3596be22fd6b6a1096c067a46b360ec7dcac963b47759213305028d60c0
  Stored in directory: /root/.cache/pip/wheels/b5/c1/bb/26ba70fb9d10f195249ef4e170a92ae83e7534e55b67786fd9
Successfully built pycuda pytools


In [ ]:
import pycuda.driver as drv
drv.init()
print(f' Devices detected: {drv.Device.count()}')

 Devices detected: 1


In [ ]:
gpu_device = drv.Device(0)
print(f'GPU: {gpu_device.name()}')
print(f"""Compute Capability: {gpu_device.compute_capability()} 
Global Memory Size: {gpu_device.total_memory()//1024**2}""")

GPU: Tesla T4
Compute Capability: (7, 5) 
Global Memory Size: 15109


Загрузка изображения и инвертирование цветов пикселей на CPU

In [4]:
from PIL import Image
import numpy as np
from google.colab import drive
import cv2
from time import time
drive.mount('/content/gdrive')
 

#Создания массива с инвертированными пикселями
img = Image.open(r"/content/gdrive/My Drive/1024x768.jpg")
img_array = np.array(img)
#Тип тот же самый, что и в IO
I_max = 255
t1 = time()
img_array = I_max - img_array
t2 = time()
print(f'Elapsed time (CPU) invertion = {round(t2-t1,7)} secs.')
inverted_img = Image.fromarray(img_array)
inverted_img.save(r"/content/gdrive/My Drive/negative_cpu_1024x768.jpg")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Elapsed time (CPU) invertion = 0.0008149 secs.


Увеличение контрастности пикселей на CPU

In [6]:
img = Image.open(r"/content/gdrive/My Drive/negative_cpu_1024x768.jpg")
img_array = np.array(img)

#Попытка сделать через сложения каждого контраста каждого пикселя
contrast_img_array = img_array.astype(np.int32)

t1 = time()
for i in range(1, len(img_array) - 1, 1):
  for j in range(1, len(img_array[0]) - 1, 1):
    for m in range(1, 3):
      contrast_img_array[i, j, m] = img_array[i, j, m] * 5 - img_array[i - 1, j, m] - img_array[i + 1, j, m] - img_array[i, j - 1, m] - img_array[i, j + 1, m]
      if contrast_img_array[i, j, m] < 0:
        contrast_img_array[i, j, m] = 0
      if contrast_img_array[i, j, m] > 255:
        contrast_img_array[i, j, m] = 255
t2 = time()
print(f'Elapsed time (CPU) sharpen = {round(t2-t1,7)} secs.')

contrast_img = Image.fromarray(contrast_img_array.astype(np.uint8))
 
contrast_img.save(r"/content/gdrive/My Drive/sharpen_negative_cpu_1024x768.jpg")
#Конец попытки


Elapsed time (CPU) sharpen = 12.3445568 secs.


Загрузка изображения и инвертирование цветов пикселей на GPU с pycuda

In [7]:
import numpy as np
from PIL import Image
from google.colab import drive
import pycuda.autoinit
from pycuda import gpuarray
from pycuda.compiler import SourceModule
from time import time
drive.mount('/content/gdrive')

invertion_image = SourceModule("""
__global__ void invertion_image(int *in, int *out)
{
  int i = blockDim.x * blockIdx.x + threadIdx.x;
  out[i] = 255 - in[i];
}
""")

#Создания массива с инвертированными пикселями
img = Image.open(r"/content/gdrive/My Drive/1024x768.jpg")
img_array = np.array(img)
input_img = img_array.astype(np.int32)

image_size = input_img.shape[0]*input_img.shape[1]*input_img.shape[2]
block_x_size = 1024
block_y_size = 1
block_z_size = 1
grid_x_size = round(image_size/block_x_size)
grid_y_size = 1
grid_z_size = 1

invertion_image_gpu_kernel = invertion_image.get_function('invertion_image')
size = input_img.shape[0]*input_img.shape[1]
host_input_array = input_img.reshape(image_size)

input_device_array = gpuarray.to_gpu(host_input_array)
output_device_array = gpuarray.empty_like(input_device_array)
t1 = time()

invertion_image_gpu_kernel(input_device_array,output_device_array,
                          block=(block_x_size,block_y_size,block_z_size),
                          grid=(grid_x_size,grid_y_size,grid_z_size))
t2 = time()
print(f'Elapsed time (GPU) invertion = {round(t2-t1,7)} secs.')
output_host_array = output_device_array.get()
output_host_array = output_host_array.astype(np.uint8)
output_host_array = output_host_array.reshape(img_array.shape)
inverted_img = Image.fromarray(output_host_array)
inverted_img.save(r"/content/gdrive/My Drive/negative_gpu_1024x768.jpg")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Elapsed time (GPU) invertion = 0.0002306 secs.


Увеличение контрастности пикселей на GPU с pycuda

In [9]:
#Увеличение контраста
import numpy as np
from PIL import Image
from google.colab import drive
import pycuda.autoinit
from pycuda import gpuarray
from pycuda.compiler import SourceModule
from time import time
drive.mount('/content/gdrive')

sharpen_image_kernel = SourceModule("""
__global__ void sharpen_image_kernel(int* imageSource, int* contrastedImage, int width)
{
    int pixelOffset = 3;
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    int middle = imageSource[i];
    int left = imageSource[i - pixelOffset];
    int right = imageSource[i + pixelOffset];
    int top = imageSource[i - (pixelOffset * width)];
    int bottom = imageSource[i + (pixelOffset * width)];
    int newValue = (middle * 5) + (left * -1) + (right * -1) + (top * -1) + (bottom * -1);
    if(newValue > 255)
    {
      newValue = 255;
    }
    else if(newValue < 0)
    {
      newValue = 0;
    }
    contrastedImage[i] = newValue;
}
""")

img = Image.open(r"/content/gdrive/My Drive/negative_gpu_1024x768.jpg")
img_array = np.array(img)
input_img = img_array.astype(np.int32)

image_size = input_img.shape[0]*input_img.shape[1]*input_img.shape[2]
block_x_size = 1024
block_y_size = 1
block_z_size = 1
grid_x_size = round(image_size/block_x_size)
grid_y_size = 1
grid_z_size = 1

sharpen_image_gpu = sharpen_image_kernel.get_function('sharpen_image_kernel')

host_input_array = input_img.reshape(image_size)

input_device_array = gpuarray.to_gpu(host_input_array)
output_device_array = gpuarray.empty_like(input_device_array)

t1 = time()
sharpen_image_gpu(input_device_array,output_device_array, np.int32(input_img.shape[1]),
                      block=(block_x_size,block_y_size,block_z_size),
                      grid=(grid_x_size,grid_y_size,grid_z_size))
t2 = time()
print(f'Elapsed time (GPU) sharpen = {round(t2-t1,7)} secs.')

output_host_array = output_device_array.get()
output_host_array = output_host_array.astype(np.uint8)
output_host_array = output_host_array.reshape(img_array.shape)
contrast_img = Image.fromarray(output_host_array)
 
contrast_img.save(r"/content/gdrive/My Drive/sharpen_negative_gpu_1024x768.jpg")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Elapsed time (GPU) sharpen = 0.0001681 secs.


/usr/local/lib/python3.8/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: module in out-of-thread context could not be cleaned up
  globals().clear()
/usr/local/lib/python3.8/dist-packages/google/colab/_variable_inspector.py:27: UserWarning: device_allocation in out-of-thread context could not be cleaned up
  globals().clear()
